In [177]:
import pandas as pd
from meteostat import Stations, Point, Monthly, Daily
from datetime import date, datetime
import time
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [170]:
df = pd.read_csv("ficheros/03-areas_country.csv", index_col=0)
df.head(10)

,latitude,longitude,town,region,country,price,water,wc,grey_water
0,39.06956,-0.40980,Carcaixent,Comunitat Valenciana,España,Free,yes,yes,yes
1,42.64159,-8.89696,Boiro,Galicia,España,Free,yes,yes,unknown
2,42.90715,0.35921,Arreau,Occitanie,France,Paying,yes,no,yes
3,38.39479,-0.41007,el Campello,Comunitat Valenciana,España,Paying,yes,yes,unknown
4,43.37179,-8.44443,A Coruña,Galicia,España,Free,yes,yes,unknown
5,40.72556,0.72158,Deltebre,Catalunya,España,Free,yes,no,unknown
6,41.81417,2.52043,Arbúcies,Catalunya,España,Free,yes,no,unknown
7,39.75402,9.67130,Cardedu,Sardigna/Sardegna,Italia,Paying,yes,yes,unknown
8,41.78121,1.27773,Guissona,Catalunya,España,Free,yes,yes,unknown
9,51.28586,4.50299,Brasschaat,Antwerpen,België / Belgique / Belgien,Free,yes,no,unknown


Temperatura media por mes del 1-1-2000 al 31-12-2022

In [287]:
def weather_month(dataf):
    
    lista_col = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
                7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"}

    data_list = []
    for ind, row in dataf.iterrows():
        lat = row["latitude"]
        lon = row["longitude"]
        city = row["town"]
        country = row["country"]

        start = datetime(2000, 1, 1)
        end = datetime(2022, 12, 31)

        stations = Stations()
        stations = stations.nearby(lat,lon)
        try:
            station = stations.fetch(4)
            station = station[station["daily_end"] > "2002-01-01"]
            station_name = station.index[0]
        except:
            station = stations.fetch(4)
            station_name = station.index[0]

        data = Daily(station_name, start, end)
        data = data.fetch()
        data.reset_index(inplace=True)
        data["time"] = pd.to_datetime(data["time"])
        data["month"] = data["time"].dt.month

        t_media = round(data.groupby("month")["tavg"].sum()/data["month"].value_counts(),2)
        df_temp = t_media.to_frame().rename_axis(None, axis=0).T
        
        df_temp["latitude"] = lat
        df_temp["longitude"] = lon
        df_temp["town"] = city
        df_temp["country"] = country
        data_list.append(df_temp)
        
    dat = pd.concat(data_list, axis=0, ignore_index= True)
    dat.rename(columns=lista_col, inplace= True)

    return dat

In [288]:
df_weather = weather_month(df)

In [256]:
df_weather.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude,town,country
0,2.36,2.75,2.86,3.32,4.17,5.05,6.38,6.95,5.86,5.09,3.91,3.35,39.06956,-0.40980,Carcaixent,España
1,9.41,9.92,11.31,13.10,15.54,17.60,19.63,19.32,17.78,15.07,11.48,9.86,42.64159,-8.89696,Boiro,España
2,5.22,6.17,8.84,11.27,14.62,18.40,19.91,19.96,17.30,13.86,8.55,6.17,42.90715,0.35921,Arreau,France
3,11.41,12.96,14.20,16.21,20.07,23.99,26.89,26.89,24.03,19.88,15.27,12.87,38.39479,-0.41007,el Campello,España
4,10.81,10.98,11.98,12.97,15.09,17.31,18.92,19.39,18.50,16.41,13.11,11.65,43.37179,-8.44443,A Coruña,España


In [289]:
df_weather.isnull().sum()

Jan          0
Feb          0
Mar          0
Apr          0
May          0
Jun          0
Jul          0
Aug          0
Sep          0
Oct          0
Nov          0
Dec          0
latitude     0
longitude    0
town         0
country      0
dtype: int64

In [290]:
df_weather.to_csv("ficheros/04-weather_coordinates.csv")

In [291]:
def prcp_month(dataf):
    
    lista_col = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
                7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"}

    data_list = []
    for ind, row in dataf.iterrows():
        lat = row["latitude"]
        lon = row["longitude"]
        city = row["town"]
        country = row["country"]

        start = datetime(2000, 1, 1)
        end = datetime(2022, 12, 31)

        stations = Stations()
        stations = stations.nearby(lat,lon)
        try:
            station = stations.fetch(4)
            station = station[station["daily_end"] > "2002-01-01"]
            station_name = station.index[0]
        except:
            station = stations.fetch(4)
            station_name = station.index[0]

        data = Daily(station_name, start, end)
        data = data.fetch()
        data.reset_index(inplace=True)
        data["time"] = pd.to_datetime(data["time"])
        data["month"] = data["time"].dt.month

        # if data["tavg"].empty:
        #     station_name = station.index[1]
        #     data = Monthly(station_name, start, end)
        #     data = data.fetch()
        #     data.reset_index(inplace=True)
        #     data["time"] = pd.to_datetime(data["time"])
        #     data["month"] = data["time"].dt.month

        t_media = round(data.groupby("month")["prcp"].sum()/data["month"].value_counts(),2)
        df_temp = t_media.to_frame().rename_axis(None, axis=0).T
        
        df_temp["latitude"] = lat
        df_temp["longitude"] = lon
        df_temp["town"] = city
        df_temp["country"] = country
        data_list.append(df_temp)
        
    dat = pd.concat(data_list, axis=0, ignore_index= True)
    dat.rename(columns=lista_col, inplace= True)

    return dat

In [292]:
df_precp = prcp_month(df)

In [294]:
df_precp.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude,town,country
0,1.16,0.94,1.65,1.35,1.57,0.89,0.23,0.55,2.34,2.22,2.09,1.20,39.06956,-0.40980,Carcaixent,España
1,6.03,4.46,4.71,4.79,3.13,2.03,1.29,1.64,2.22,7.01,7.15,7.00,42.64159,-8.89696,Boiro,España
2,3.35,2.76,2.52,3.08,3.15,2.33,1.98,1.84,2.21,2.17,3.63,2.85,42.90715,0.35921,Arreau,France
3,1.09,0.23,2.01,1.62,0.62,0.31,0.13,0.60,1.33,1.13,0.82,0.81,38.39479,-0.41007,el Campello,España
4,3.89,3.30,2.95,2.82,1.90,1.71,0.98,1.19,1.40,4.19,5.12,4.03,43.37179,-8.44443,A Coruña,España
